In [1]:
import deepxde as dde
import numpy as np

import yaml
import pandas as pd
import time
import datetime
import os

#from deepxde.backend import tf
#from deepxde.backend import torch
import torch
from scipy.special import gamma
from scipy.special import lambertw

#import mayavi.mlab as mlab
import matplotlib.pyplot as plt

#from draw_counter import random_sphere_points, uniform_sample_points
from PIL import Image

torch.set_default_tensor_type(torch.cuda.FloatTensor)

CopyFlag = 0
## 读取Config文件
with open('./Config/train_20220715-Var.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)
config['NetWidth']

## 更新参数
## 单位统一至cm，定义各个参数
# D = 0.585
# Rsd = 17.5573
# #Rwall = 2.654
# Rwall = 2.56401
# rpump = 0.75
# q = 5
# rcell = 1.5

# Rrel = Rsd + q*Rwall
# I = config['LightIntensity']*1e-3*1e4 # kg*(cm)^2/s^3

# k1 = 10**(-20)*10**8 # kg*cm^4/s^2
# k2 = 10**(-17)*10**4 # cm^2
# k3 = 1 # cm^-3
# sigm = 2.34601*10**(-13) # cm^2

# n = k3*(1/(T+273.15))*10**(21.866+4.402-4453/(T+273.15))

# D = 6.9519E-1 #cm²/s
# I = 25*10 # kg*(cm)^2/s^3
# q = 5
# Rwall = 6.4424 #1/s
# Rsd = 13.457 #1/s
# Rrel = Rsd+q*Rwall
# T = config['Temperature']

# k1 = 10**(-20)*10**8 # kg*cm^4/s^2
# k2 = 10**(-17)*10**4 # cm^2
# k3 = 1 # cm^-3
# sigm = 2.34601*10**(-13) # cm^2
# n = 3.7405E13 #1/m³
# rcell = 1.5
# rpump = 0.75

Using backend: pytorch



{'ConfigDate': datetime.date(2022, 7, 27), 'CSVPath': './result/Variant/TrainResult0727', 'CSVRoot': './result/Variant/', 'LightIntensity': 25, 'Temperature': 170, 'CopyFlag': [1, 2, 3], 'NetDepth': [4], 'NetWidth': [120], 'TrainEpoch': 90000, 'Rsd': [11.505, 12.448, 14.019, 12.526, 13.457, 15.017], 'Rwall': [10.379, 10.826, 11.285, 6.1763, 6.4424, 6.716], 'Rrel': [63.398, 66.576, 70.446, 43.408, 45.669, 48.597], 'D': [0.87449, 0.89313, 0.9119, 0.68068, 0.69519, 0.7098], 'n': ['2.2432E13', '3.7405E13', '6.0952E13', '2.2432E13', '3.7405E13', '6.0952E13']}


[120]

In [2]:
## 前向实验参数
Phe = [1.6 , 2.1] # atm
Temp = [160, 170, 180]
I = 25*10 # kg*(cm)^2/s^3
q = 5

# Rsd = [11.505 12.448  14.019  12.526  13.457  15.017 ]
# Rwall = [10.379 10.826  11.285  6.1763  6.4424  6.716 ]
# Rrel = [63.398 66.576  70.446  43.408  45.669  48.597 ]

# D = [8.7449E-1 8.9313E-1  9.119E-1 6.8068E-1 6.9519E-1 7.098E-1]

# n = [2.2432E13 3.7405E13 6.0952E13 2.2432E13 3.7405E13 6.0952E13]



k1 = 10**(-20)*10**8 # kg*cm^4/s^2
k2 = 10**(-17)*10**4 # cm^2
k3 = 1 # cm^-3
rcell = 1.5
rpump = 0.75
sigm = 2.34601*10**(-13) # cm^2

In [3]:
'''
读取Ref文件，COMSOL仿真结果
'''
# 读取3D仿真结果文件
#filename_3D = './COMSOL_data/COMSOL_3D.csv'
def read_COMSOL(filename_3D):
    #df = pd.read_csv(filename, header=9, names=['x', 'y', 'z', 'Pz'])
    df_3D = pd.read_csv(filename_3D, header=9, names=['x', 'y', 'z', 'Pz'])
    axis_3D_x = np.asarray(df_3D['x'])
    axis_3D_y = np.asarray(df_3D['y'])
    axis_3D_z = np.asarray(df_3D['z'])
    axis_3D_Pz = np.asarray(df_3D['Pz'])
    axis_3D_axis = np.stack((axis_3D_x, axis_3D_y, axis_3D_z), axis=1)
    return axis_3D_axis, axis_3D_Pz
# # 读取z-axis实验结果
# filename = './COMSOL_data/COMSOL_zaxis.csv'
# #df = pd.read_csv(filename, header=9, names=['x', 'y', 'z', 'Pz'])
# df = pd.read_csv(filename, header=9, names=['Pz'])
# ZAxis_COMSOL = np.asarray(df['Pz'])



# ## PINNs，计算z-axis的极化率数据. z_fitness 拟合结果

# def Z_Axis(sample_num=300, rcell=1.5):
#     x = 0
#     y = 0
#     z_sequence = np.linspace(-rcell, rcell, sample_num)
#     sample_data = []
    
#     for z in z_sequence:
#         sample_data.append((x, y, z))
        
#     sample_data = np.asarray(sample_data, dtype=np.float32)
#     return sample_data


# def fitness_metric(y_pred, y_ref):
#     # 优化度拟合矩阵
#     y_l2_error = np.linalg.norm(y_pred - y_ref, ord=2) # L2 范数
#     y_pred_sum = np.sum(y_pred**2)
    
#     fitness = 1 - np.sqrt(y_l2_error / y_pred_sum)
#     return fitness

def bloch_pde(x_in, y_in):
    ''' x_in: (x,y,z) axis
        y_in: Pz polarization
    '''
    x_in_copy = x_in.detach().cpu().numpy()
    x = x_in_copy[:, 0:1]
    y = x_in_copy[:, 1:2]
    z = x_in_copy[:, 2:3]
    
#     sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
#     x_arr = x.eval(session=sess)
    dy_xx = dde.grad.hessian(y_in, x_in, i=0, j=0)
    dy_yy = dde.grad.hessian(y_in, x_in, i=1, j=1)
    dy_zz = dde.grad.hessian(y_in, x_in, i=2, j=2) # dy/dxidxj
    
    plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
    plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
    Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2))
    Rop0_z = np.asarray(Rop0_z)    
    
    Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    Rop = torch.from_numpy(Rop).cuda()
    #Rop = Rop0
    
    return -D*(dy_xx+dy_yy+dy_zz) + ((Rop+Rsd)/q+Rwall)*y_in - Rop/q


def func_boundary(x_in, Pz, numpy_X):
    x_in_copy = x_in.detach().cpu().numpy()
    x = x_in_copy[:, 0:1]
    y = x_in_copy[:, 1:2]
    z = x_in_copy[:, 2:3]
    
    dPz_x = dde.grad.jacobian(Pz, x_in, i=0, j=0) # This is right
    dPz_y = dde.grad.jacobian(Pz, x_in, i=0, j=1)
    dPz_z = dde.grad.jacobian(Pz, x_in, i=0, j=2)
    
    #Rop = Rop0 * np.exp(-2*(x**2+y**2)/rpump**2)
    
    plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
    plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
    Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2))
    Rop0_z = np.asarray(Rop0_z)      
    
    Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    
    second_term = np.sqrt(1/2*np.abs(Rop*D))
    
    # tranfer back to torch.tensor
    second_term = torch.from_numpy(second_term).cuda()
    Rop = torch.from_numpy(Rop).cuda()
    x = torch.from_numpy(x).cuda()
    y = torch.from_numpy(y).cuda()
    z = torch.from_numpy(z).cuda()
    
    return (D/rcell*(x*dPz_x + y*dPz_y + z*dPz_z) + Pz*second_term)

In [4]:
## 自定义callbacks, 每1000epoch，打印变量的值

from deepxde.callbacks import Callback
import os
import torch

class printVariable(Callback):
    def __init__(self, var_list, period=1000, filename=None, precision=2):
        super().__init__()
        self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.period = period
        self.precision = precision

        #self.file = sys.stdout if filename is None else open(filename, "w", buffering=1)
        self.value = None
        self.epochs_since_last = 0
        self.model = None
    
    def on_train_begin(self):

        self.value = [var.detach().item() for var in self.var_list]
        print(
            'Variable pred',
            self.value
        )
    
    def on_epoch_end(self):
        self.epochs_since_last += 1
        if self.epochs_since_last >= self.period:
            self.epochs_since_last = 0
            self.on_train_begin()

    def get_value(self):
        """Return the variable values."""
        return self.value
    
    def set_model(self, model):
        if model is not self.model:
            self.model = model
            self.init()
   
class saveModel(Callback):
    def __init__(self, saveDir, saveName, period=1000, filename=None, precision=2):
        super().__init__()
        #self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.period = period
        self.precision = precision

        #self.file = sys.stdout if filename is None else open(filename, "w", buffering=1)
        self.value = None
        self.epochs_since_last = 0
        self.model = None
        self.modelNum = 0
        self.saveDir = saveDir
        self.saveName = saveName
        
    def set_model(self, model):
        if model is not self.model:
            self.model = model
            self.init()    
            
    def on_epoch_end(self):
        self.epochs_since_last += 1
        if self.epochs_since_last >= self.period:
            self.modelNum += 1
            self.epochs_since_last = 0  
            if not os.path.exists(self.saveDir):
                #os.mkdir(self.saveDir)
                os.makedirs(self.saveDir)
            savePath = os.path.join(self.saveDir, self.saveName+str(self.modelNum)+'.pt')
            #print(self.model.net.state_dict())
            checkpoint = {
                "model_state_dict": self.model.net.state_dict(),
                "optimizer":self.model.opt.state_dict()
            }
            torch.save(checkpoint, savePath)            

class loadModel(Callback):
    def __init__(self, loadPath):
        super().__init__()
        #self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.loadPath = loadPath
        
    def set_model(self, model):
        if model is not self.model:
            self.model = model
            self.init()    
            
    def on_train_begin(self):
        checkpoint = torch.load(self.loadPath)
        self.model.net.load_state_dict(checkpoint["model_state_dict"])
        self.model.opt.load_state_dict(checkpoint["optimizer"])
        print('Load model successful')
        
class EarlyStoping(Callback):
    def __init__(self,sigma=5e-4):
        super().__init__()
        #self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.epochs_since_last = 0
        self.sigma = sigma
        
    def on_epoch_end(self):
        self.epochs_since_last += 1
        train_loss = sum(self.model.train_state.loss_train)
        if train_loss < self.sigma:
            self.model.stop_training = True
            print('Early stoping')
    

In [5]:
## 模块化

def train_process(param_dict):
    
    Rrel = param_dict['Rrel']
    Rwall = param_dict['Rwall']
    n = param_dict['n']
    Rsd = param_dict['Rsd']
    D = param_dict['D']
    SaveName = param_dict['SaveName']
    COMSOL_file = param_dict['COMSOL_file']
    
    
    geom = dde.geometry.Sphere([0, 0, 0], rcell)
    bc = dde.icbc.OperatorBC(geom, func_boundary, lambda _, on_boundary: on_boundary) # 函数调研下，operatorBC

    data = dde.data.PDE(geom, bloch_pde, bc, num_domain=20000, num_boundary=2000)
    ModelSaveDir = './model/forward/'
    if not os.path.exists(ModelSaveDir):
        os.makedirs(ModelSaveDir)
    ## 核心训练程序
    for depth in config['NetDepth']:
        for width in config['NetWidth']:
            net = dde.nn.FNN([3] + [width] * depth + [1], "tanh", "Glorot normal")
            # net.apply_output_transform(
            #     lambda x, y: (1 - tf.reduce_sum(x ** 2, axis=1, keepdims=True)) * y
            # )
            myCallback1 = saveModel(ModelSaveDir,SaveName,period=10000)
            myCallback2 = EarlyStoping()
            model = dde.Model(data, net)
            model.compile("adam", lr=1e-4)
            #losshistory, train_state = model.train(epochs=8000, model_save_path='Bloch_Rop_v1')
            losshistory, train_state = model.train(epochs=config['TrainEpoch'], callbacks=[myCallback1, myCallback2])
            print('Train finish : {0} width {1} depth'.format(width, depth))

            YZ_dir_path = './result_Forward/{0}_NetDepth{1}_Width{2}_copy{3}'.format(SaveName, depth,\
                                                                                        width,config['CSVPath'],CopyFlag)
            if not os.path.exists(YZ_dir_path):
                os.makedirs(YZ_dir_path)
    #         zAxis_pred = model.predict(Z_Axis())[:, 0]
    #         zAxis_df = pd.DataFrame(zAxis_pred)
    #         zAxis_path = os.path.join(YZ_dir_path, 'zAxis_pred.csv')
    #         zAxis_df.to_csv(zAxis_path)       
    #         z_fitness = fitness_metric(zAxis_pred, ZAxis_COMSOL)

            ## 计算三维数据预测结果
        
            axis_3D_axis, axis_3D_Pz = read_COMSOL(COMSOL_file)
            y_pred_3D = model.predict(axis_3D_axis)[:, 0]
            y_ref = axis_3D_Pz

            #L2_mean = np.linalg.norm(y_pred_3D - y_ref, ord=2) # 开了跟号
            L2_error = np.mean(np.sqrt((y_pred_3D-y_ref)**2))
            print('Evaluation : ', L2_error)
            ## 按照实验时间，保存实验结果
            date_time =  datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') # 2022-06-18 13:49
            CSVRoot = config['CSVRoot']
            if not os.path.exists(CSVRoot):
                os.makedirs(CSVRoot)
            record_path = config['CSVPath'] + str(config['CopyFlag'][CopyFlag]) + '.csv'

            # if not os.path.exists(record_path):
            #     os.mkdir('./result/')

            EvaluateDf = pd.DataFrame({
                'Date': [date_time],
                'SaveName': [SaveName],
                'Epoch': [config['TrainEpoch']],
                'Intensity(mW)': [config['LightIntensity']],
                'Temperature' : [config['Temperature']],
                'Network depth' : [depth],
                'Network width' : [width],
                #'Z axis fitness' : [z_fitness],
                'L2 mean error': [L2_error]
            })    

            EvaluateDf.to_csv(record_path, index=False, mode='a')

            ## 保存截面数据, X=0的YZ平面，坐标轴生成

            sample_num = 200

            def plane_YZ(x=0, sample_num = 200):
                radius_x = np.sqrt(rcell**2 -x**2)
                sample_data = []    

                for theta in np.linspace(0, 2*np.pi, sample_num):
                    for r_pow in np.linspace(0, radius_x**2, sample_num):
                        y = np.sqrt(r_pow) * np.cos(theta)
                        z = np.sqrt(r_pow) * np.sin(theta)
                        sample_data.append((x, y, z)) # 200*200 = 40000个采样点

                sample_data = np.asarray(sample_data, dtype=np.float32
                                        )
                return sample_data

            YZ_axis = plane_YZ(0)
            y_pred_YZ = model.predict(YZ_axis)[:, 0]
            y_pred_YZ = np.reshape(y_pred_YZ, (sample_num, sample_num))
            YZ_df = pd.DataFrame(y_pred_YZ.T)
            YZ_file_path = os.path.join(YZ_dir_path, 'Plane_YZ_Epoch_{}.csv'.format(config['TrainEpoch']))    
            YZ_df.to_csv(YZ_file_path)  
            

In [6]:

SaveNameList = ['He16Temp160', 'He16Temp170','He16Temp180','He21Temp160','He21Temp170','He21Temp180']
COMSOLFileList = ['COMSOL_He16_Temp160.csv','COMSOL_He16_Temp170.csv','COMSOL_He16_Temp180.csv','COMSOL_He21_Temp160.csv','COMSOL_He21_Temp170.csv',\
                 'COMSOL_He21_Temp180.csv']
COMSOLRoot = './COMSOL_data/variation_exp/'

for i in range(6):
    Rrel = config['Rrel'][i]
    Rwall = config['Rwall'][i]
    Rsd = config['Rsd'][i]
    D = config['D'][i]
    n =config['n'][i]
    n = float(n)
    SaveName = SaveNameList[i]
    COMSOL_file = os.path.join(COMSOLRoot, COMSOLFileList[i])
    
    paramDict = {'Rrel':Rrel,
                'Rwall':Rwall,
                'Rsd':Rsd,
                'D':D,
                'n':n,
                'SaveName':SaveName,
                'COMSOL_file':COMSOL_file}
    train_process(paramDict)



/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+20002=20002. 
  total_n_samples))
/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+2002=2002. 
  total_n_samples))


Compiling model...
'compile' took 0.000282 s

Training model...

Step      Train loss              Test loss               Test metric
0         [4.54e+01, 4.83e-02]    [4.54e+01, 4.83e-02]    []  
1000      [1.85e-01, 1.08e+00]    [1.85e-01, 1.08e+00]    []  
2000      [1.46e-01, 9.66e-01]    [1.46e-01, 9.66e-01]    []  
3000      [1.47e-01, 8.77e-01]    [1.47e-01, 8.77e-01]    []  
4000      [1.68e-01, 6.63e-01]    [1.68e-01, 6.63e-01]    []  
5000      [1.36e-01, 2.15e-01]    [1.36e-01, 2.15e-01]    []  
6000      [4.10e-02, 1.97e-02]    [4.10e-02, 1.97e-02]    []  
7000      [1.77e-02, 5.49e-03]    [1.77e-02, 5.49e-03]    []  
8000      [9.04e-03, 3.36e-03]    [9.04e-03, 3.36e-03]    []  
9000      [5.36e-03, 2.65e-03]    [5.36e-03, 2.65e-03]    []  
10000     [3.70e-03, 2.13e-03]    [3.70e-03, 2.13e-03]    []  
11000     [9.90e-03, 1.82e-03]    [9.90e-03, 1.82e-03]    []  
12000     [2.20e-03, 1.36e-03]    [2.20e-03, 1.36e-03]    []  
13000     [1.85e-03, 1.10e-03]    [1.85e-03, 1

/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+20002=20002. 
  total_n_samples))


Compiling model...
'compile' took 0.000197 s

Training model...

Step      Train loss              Test loss               Test metric
0         [1.28e+02, 1.27e-01]    [1.28e+02, 1.27e-01]    []  


/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+2002=2002. 
  total_n_samples))


...

1000      [3.25e-01, 1.73e+00]    [3.25e-01, 1.73e+00]    []  
2000      [2.13e-01, 1.59e+00]    [2.13e-01, 1.59e+00]    []  
3000      [2.22e-01, 1.44e+00]    [2.22e-01, 1.44e+00]    []  
4000      [2.88e-01, 1.01e+00]    [2.88e-01, 1.01e+00]    []  
5000      [1.04e-01, 7.14e-02]    [1.04e-01, 7.14e-02]    []  
6000      [3.14e-02, 9.26e-03]    [3.14e-02, 9.26e-03]    []  
7000      [1.38e-02, 4.13e-03]    [1.38e-02, 4.13e-03]    []  
8000      [7.58e-03, 2.62e-03]    [7.58e-03, 2.62e-03]    []  
9000      [4.88e-03, 1.72e-03]    [4.88e-03, 1.72e-03]    []  
10000     [3.61e-03, 1.19e-03]    [3.61e-03, 1.19e-03]    []  
11000     [2.91e-03, 8.74e-04]    [2.91e-03, 8.74e-04]    []  
12000     [2.45e-03, 6.80e-04]    [2.45e-03, 6.80e-04]    []  
13000     [2.10e-03, 5.48e-04]    [2.10e-03, 5.48e-04]    []  
14000     [1.83e-03, 4.52e-04]    [1.83e-03, 4.52e-04]    []  
15000     [1.61e-03, 3.80e-04]    [1.61e-03, 3.80e-04]    []  
16000     [1.44e-03, 3.23e-04]    [1.44e-03, 3.23e-04] 

/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+20002=20002. 
  total_n_samples))


Compiling model...
'compile' took 0.000241 s

Training model...

Step      Train loss              Test loss               Test metric
0         [2.06e+02, 3.62e-01]    [2.06e+02, 3.62e-01]    []  


/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+2002=2002. 
  total_n_samples))


1000      [9.50e-01, 4.21e+00]    [9.50e-01, 4.21e+00]    []  
2000      [5.43e-01, 3.88e+00]    [5.43e-01, 3.88e+00]    []  
3000      [5.54e-01, 3.61e+00]    [5.54e-01, 3.61e+00]    []  
4000      [5.69e-01, 3.22e+00]    [5.69e-01, 3.22e+00]    []  
5000      [5.57e-01, 2.18e+00]    [5.57e-01, 2.18e+00]    []  
6000      [3.08e-01, 1.40e+00]    [3.08e-01, 1.40e+00]    []  
7000      [3.24e-01, 9.49e-01]    [3.24e-01, 9.49e-01]    []  
8000      [1.89e-01, 2.56e-01]    [1.89e-01, 2.56e-01]    []  
9000      [6.19e-02, 1.79e-02]    [6.19e-02, 1.79e-02]    []  
10000     [3.61e-02, 6.01e-03]    [3.61e-02, 6.01e-03]    []  
11000     [3.81e-02, 3.65e-03]    [3.81e-02, 3.65e-03]    []  
12000     [1.69e-02, 2.36e-03]    [1.69e-02, 2.36e-03]    []  
13000     [1.32e-02, 1.79e-03]    [1.32e-02, 1.79e-03]    []  
14000     [1.08e-02, 1.42e-03]    [1.08e-02, 1.42e-03]    []  
15000     [9.02e-03, 1.18e-03]    [9.02e-03, 1.18e-03]    []  
16000     [7.75e-03, 1.01e-03]    [7.75e-03, 1.01e-03] 

In [7]:
float(config['n'][1])

37405000000000.0